<a href="https://colab.research.google.com/github/abhaykatheria/Deep-Learning-colab-notebooks/blob/master/CHH-OLA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [0]:
!pip install fastai==0.7.0
from fastai.imports import*
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics

In [0]:
!pip install PyDrive

     |████████████████████████████████| 993kB 2.7MB/s 
  Created wheel for PyDrive: filename=PyDrive-1.3.1-cp36-none-any.whl size=27437 sha256=20d7c67f8d43bdb930bcd80a3b948628cd04c21c7b7a23dce146a2b238556560
  Stored in directory: /root/.cache/pip/wheels/fa/d2/9a/d3b6b506c2da98289e5d417215ce34b696db856643bad779f4
Successfully built PyDrive


In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
 
download = drive.CreateFile({'id': '1jke6L2f4idME-AQv3nH9faZvT_qw_Q0s'})
download.GetContentFile('xtrain.csv')

In [0]:
download = drive.CreateFile({'id': '1iHnnVM0wkZMU1vFQs9cK_xkJpQCedjbM'})
download.GetContentFile('xtest.csv')

In [0]:
download = drive.CreateFile({'id': '1ScYIUrAjEhzk3SMsaBFZHF5KHxH_cYVm'})
download.GetContentFile('ytrain.csv')

In [0]:
xtrain =pd.read_csv("xtrain.csv")

In [0]:
xtrain.head()

,ID,vendor_id,pickup_loc,drop_loc,driver_tip,mta_tax,distance,pickup_time,drop_time,num_passengers,toll_amount,payment_method,rate_code,stored_flag,extra_charges,improvement_charge,total_amount
0,0,1,170,233,1.83,0.5,0.70,04-04-2017 17:59,04-04-2017 18:05,1,0.0,1,1,N,1.0,0.3,9.13
1,1,2,151,243,3.56,0.5,4.64,04-03-2017 19:03,04-03-2017 19:20,1,0.0,1,1,N,1.0,0.3,21.36
2,2,2,68,90,1.50,0.5,1.29,04-03-2017 15:06,04-03-2017 15:12,2,0.0,1,1,N,0.0,0.3,8.80
3,3,2,142,234,1.50,0.5,2.74,04-04-2017 08:10,04-04-2017 08:27,1,0.0,1,1,N,0.0,0.3,14.80
4,4,2,238,238,0.00,0.5,0.45,04-05-2017 14:02,04-05-2017 14:05,6,0.0,2,1,N,0.0,0.3,4.80


In [0]:
ytrain = pd.read_csv("ytrain.csv")

In [0]:
ytrain.head()

,Unnamed: 0,0
0,0,9.13
1,1,21.36
2,2,8.80
3,3,14.80
4,4,4.80


In [0]:
ytrain=ytrain.iloc[:,1:]

In [0]:
ytrain.head()

,0
0,9.13
1,21.36
2,8.80
3,14.80
4,4.80


In [0]:
def data_preprocess(df):
    pickup = df['pickup_time'].values[np.arange(df.values.shape[0])]
    df["hour"] = pd.DataFrame([24-int(date[11:13]) if date[-1:-3]=='PM' else int(date[11:13]) for date in pickup])
    df['min'] = pd.DataFrame([int(date[14:16]) for date in pickup])
    df['day'] = pd.DataFrame([int(date[0:2]) for date in pickup])
    #df['month'] = pd.DataFrame([int(date[3:5]) for date in pickup])
    non_useful_features = ['ID','pickup_time','drop_time','stored_flag']
    df = df.drop(non_useful_features,axis=1)
    return(df)

def train_data_creation(df):
    df = data_preprocess(df)
    Y = df['total_amount'].values
    X = df.drop('total_amount',axis=1).values
    X = (X-X.mean())/X.std()
    t_X = pd.DataFrame(X)
    t_X.to_csv("X_train.csv")
    t_Y = pd.DataFrame(Y)
    t_Y.to_csv("Y_train.csv")
    

def test_data_creation(df):
    df = data_preprocess(df)
    X = df.values
    X = (X-X.mean())/X.std()
    t_X = pd.DataFrame(X)
    t_X.to_csv("X_testf.csv")
    


In [0]:
train_data_creation(xtrain)

In [0]:
X_train = pd.read_csv("X_train.csv")
Y_train = pd.read_csv("Y_train.csv")


In [0]:
X_test = pd.read_csv("xtest.csv")

In [0]:
X_train.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,-0.411406,2.397509,3.444619,-0.397611,-0.419717,-0.416393,-0.411406,-0.428027,-0.411406,-0.411406,-0.411406,-0.423041,-0.145474,0.552600,-0.361544
1,1,-0.394786,2.081713,3.610827,-0.368857,-0.419717,-0.350907,-0.411406,-0.428027,-0.411406,-0.411406,-0.411406,-0.423041,-0.112232,-0.378165,-0.361544
2,2,-0.394786,0.702187,1.067845,-0.403096,-0.419717,-0.406586,-0.394786,-0.428027,-0.411406,-0.411406,-0.428027,-0.423041,-0.178715,-0.328302,-0.361544
3,3,-0.394786,1.932126,3.461240,-0.403096,-0.419717,-0.382486,-0.411406,-0.428027,-0.411406,-0.411406,-0.428027,-0.423041,-0.295061,-0.261819,-0.361544
4,4,-0.394786,3.527723,3.527723,-0.428027,-0.419717,-0.420548,-0.328302,-0.428027,-0.394786,-0.411406,-0.428027,-0.423041,-0.195336,-0.394786,-0.361544


In [0]:
Y_train.head()

,Unnamed: 0,0
0,0,9.13
1,1,21.36
2,2,8.80
3,3,14.80
4,4,4.80


In [0]:
X_test=pd.read_csv("xtest.csv")

In [0]:
X_test.head()

,ID,vendor_id,pickup_loc,drop_loc,driver_tip,mta_tax,distance,pickup_time,drop_time,num_passengers,toll_amount,payment_method,rate_code,stored_flag,extra_charges,improvement_charge
0,0,1,68.0,186.0,1.00,0.5,0.70,04/05/2017 05:49:46 AM,04/05/2017 05:54:03 AM,1.0,0.0,1.0,1.0,N,0.5,0.3
1,1,1,170.0,161.0,2.55,0.5,1.00,04/07/2017 08:57:27 AM,04/07/2017 09:16:00 AM,1.0,0.0,1.0,1.0,N,0.0,0.3
2,2,2,13.0,246.0,2.96,0.5,3.35,04/06/2017 08:06:41 AM,04/06/2017 08:21:33 AM,2.0,0.0,1.0,1.0,N,0.0,0.3
3,3,1,170.0,233.0,2.00,0.5,0.80,04/04/2017 08:31:34 AM,04/04/2017 08:37:21 AM,2.0,0.0,1.0,1.0,N,0.0,0.3
4,4,1,237.0,236.0,1.00,0.5,0.70,04/07/2017 10:54:06 AM,04/07/2017 11:00:59 AM,1.0,0.0,1.0,1.0,Y,0.0,0.3


In [0]:
test_data_creation(X_test)

In [0]:
X_test_f = pd.read_csv("X_testf.csv")

In [0]:
X_test_f.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,-0.403674,0.708084,2.666106,-0.403674,-0.411971,-0.408652,-0.403674,-0.420268,-0.403674,-0.403674,-0.411971,-0.41529,-0.337301,0.392809,-0.353894
1,1,-0.403674,2.400611,2.251271,-0.377955,-0.411971,-0.403674,-0.403674,-0.420268,-0.403674,-0.403674,-0.420268,-0.41529,-0.287520,0.525556,-0.353894
2,2,-0.387081,-0.204553,3.661710,-0.371151,-0.411971,-0.364680,-0.387081,-0.420268,-0.403674,-0.403674,-0.420268,-0.41529,-0.287520,-0.320707,-0.353894
3,3,-0.403674,2.400611,3.445996,-0.387081,-0.411971,-0.406993,-0.387081,-0.420268,-0.403674,-0.403674,-0.420268,-0.41529,-0.287520,0.094128,-0.353894
4,4,-0.403674,3.512370,3.495776,-0.403674,-0.411971,-0.408652,-0.403674,-0.420268,-0.403674,-0.403674,-0.420268,-0.41529,-0.254334,0.475776,-0.353894


In [0]:
X_test_f = X_test_f.iloc[:,1:]

In [0]:
X_test_f.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-0.403674,0.708084,2.666106,-0.403674,-0.411971,-0.408652,-0.403674,-0.420268,-0.403674,-0.403674,-0.411971,-0.41529,-0.337301,0.392809,-0.353894
1,-0.403674,2.400611,2.251271,-0.377955,-0.411971,-0.403674,-0.403674,-0.420268,-0.403674,-0.403674,-0.420268,-0.41529,-0.287520,0.525556,-0.353894
2,-0.387081,-0.204553,3.661710,-0.371151,-0.411971,-0.364680,-0.387081,-0.420268,-0.403674,-0.403674,-0.420268,-0.41529,-0.287520,-0.320707,-0.353894
3,-0.403674,2.400611,3.445996,-0.387081,-0.411971,-0.406993,-0.387081,-0.420268,-0.403674,-0.403674,-0.420268,-0.41529,-0.287520,0.094128,-0.353894
4,-0.403674,3.512370,3.495776,-0.403674,-0.411971,-0.408652,-0.403674,-0.420268,-0.403674,-0.403674,-0.420268,-0.41529,-0.254334,0.475776,-0.353894


In [0]:
X_train = X_train.iloc[:,1:]
Y_train = Y_train.iloc[:,1:]


In [0]:
X_train.head()


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-0.411406,2.397509,3.444619,-0.397611,-0.419717,-0.416393,-0.411406,-0.428027,-0.411406,-0.411406,-0.411406,-0.423041,-0.145474,0.552600,-0.361544
1,-0.394786,2.081713,3.610827,-0.368857,-0.419717,-0.350907,-0.411406,-0.428027,-0.411406,-0.411406,-0.411406,-0.423041,-0.112232,-0.378165,-0.361544
2,-0.394786,0.702187,1.067845,-0.403096,-0.419717,-0.406586,-0.394786,-0.428027,-0.411406,-0.411406,-0.428027,-0.423041,-0.178715,-0.328302,-0.361544
3,-0.394786,1.932126,3.461240,-0.403096,-0.419717,-0.382486,-0.411406,-0.428027,-0.411406,-0.411406,-0.428027,-0.423041,-0.295061,-0.261819,-0.361544
4,-0.394786,3.527723,3.527723,-0.428027,-0.419717,-0.420548,-0.328302,-0.428027,-0.394786,-0.411406,-0.428027,-0.423041,-0.195336,-0.394786,-0.361544


In [0]:
m = RandomForestRegressor(n_jobs=-1,n_estimators=64,max_depth=16)
m.fit(X_train,Y_train)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=16,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=64, n_jobs=-1,
                      oob_score=False, random_state=None, verbose=0,
                      warm_start=False)

In [0]:
RandomForestRegressor?

In [0]:
m.score(X_train,Y_train)

0.9783508781642898

In [0]:
from sklearn.metrics import mean_squared_error

In [0]:
yp = m.predict(X_train)

In [0]:
np.unique(yp)

array([   0.     ,    0.0012 ,    0.01292, ..., 2073.7204 , 2244.18667, 2330.81125])

In [0]:
mse = mean_squared_error(yp,Y_train.values)

In [0]:
mse

4.921848631419162

In [0]:
test_data_creation(xtest)

In [0]:
xtest=xtest.iloc[:,1:]

In [0]:
xtest=xtest.values
xtest=np.delete(xtest,9,axis=1)
xtest = pd.DataFrame(xtest)

In [0]:
ypredicted = m.predict(X_test_f)

In [0]:
xtrain.iloc[0,:]

Unnamed: 0    0.000000
0            -0.397057
1             2.231409
2             3.211251
3            -0.384147
4            -0.401722
5            -0.397057
6            -0.412610
7            -0.397057
8            -0.397057
9            -0.412610
10           -0.397057
11           -0.407944
12           -0.334844
Name: 0, dtype: float64

In [0]:
ypredicted[0]

10.594309409934533

In [0]:
np.unique(ypredicted)

array([  5.12652,   5.13354,   5.14067, ..., 266.5775 , 293.76922, 317.34808])

In [0]:
oncemore1= pd.DataFrame(ypredicted)

In [0]:
oncemore1.to_csv("once_more_again_again_again.csv")

In [0]:
data_preprocess(xtrain)

In [0]:
df = pd.read_csv("df.csv")

In [0]:
Y_train.info()

In [0]:
Y_train

In [0]:
Y_train = Y_train.iloc[:,-1]
X_train = X_train.iloc[:,1:]

In [0]:
Y_train.shape

(1037198,)

In [0]:
X_test = pd.read_csv("xtest.csv")

In [0]:
X_test.head()

,ID,vendor_id,pickup_loc,drop_loc,driver_tip,mta_tax,distance,pickup_time,drop_time,num_passengers,toll_amount,payment_method,rate_code,stored_flag,extra_charges,improvement_charge
0,0,1,68.0,186.0,1.00,0.5,0.70,04/05/2017 05:49:46 AM,04/05/2017 05:54:03 AM,1.0,0.0,1.0,1.0,N,0.5,0.3
1,1,1,170.0,161.0,2.55,0.5,1.00,04/07/2017 08:57:27 AM,04/07/2017 09:16:00 AM,1.0,0.0,1.0,1.0,N,0.0,0.3
2,2,2,13.0,246.0,2.96,0.5,3.35,04/06/2017 08:06:41 AM,04/06/2017 08:21:33 AM,2.0,0.0,1.0,1.0,N,0.0,0.3
3,3,1,170.0,233.0,2.00,0.5,0.80,04/04/2017 08:31:34 AM,04/04/2017 08:37:21 AM,2.0,0.0,1.0,1.0,N,0.0,0.3
4,4,1,237.0,236.0,1.00,0.5,0.70,04/07/2017 10:54:06 AM,04/07/2017 11:00:59 AM,1.0,0.0,1.0,1.0,Y,0.0,0.3


In [0]:
X_testf = pd.read_csv("X_testf.csv")

In [0]:

X_testf = X_testf.iloc[:,1:]

In [0]:
X_testf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-0.403674,0.708084,2.666106,-0.403674,-0.411971,-0.408652,-0.403674,-0.420268,-0.403674,-0.403674,-0.411971,-0.41529,-0.337301,0.392809,-0.353894
1,-0.403674,2.400611,2.251271,-0.377955,-0.411971,-0.403674,-0.403674,-0.420268,-0.403674,-0.403674,-0.420268,-0.41529,-0.287520,0.525556,-0.353894
2,-0.387081,-0.204553,3.661710,-0.371151,-0.411971,-0.364680,-0.387081,-0.420268,-0.403674,-0.403674,-0.420268,-0.41529,-0.287520,-0.320707,-0.353894
3,-0.403674,2.400611,3.445996,-0.387081,-0.411971,-0.406993,-0.387081,-0.420268,-0.403674,-0.403674,-0.420268,-0.41529,-0.287520,0.094128,-0.353894
4,-0.403674,3.512370,3.495776,-0.403674,-0.411971,-0.408652,-0.403674,-0.420268,-0.403674,-0.403674,-0.420268,-0.41529,-0.254334,0.475776,-0.353894


In [0]:
fuck = m.predict(X_testf)

ValueError: ignored

In [0]:
fuck[2]

26.247000000000003

In [0]:
np.unique(fuck)

array([  6.697,   6.731,   6.745, ..., 245.478, 260.884, 269.268])

In [0]:
fuck = pd.DataFrame(fuck)
fuck.to_csv("fuck.csv")